In [10]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import locale
import nltk
import re
import json
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from IPython.display import display
nltk.download("stopwords")
nltk.download("punkt")
locale.setlocale( locale.LC_ALL, 'es_ES.UTF-8') 
plt.rcParams["figure.figsize"] = (16,9)
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to /Users/tim/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/tim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [11]:
propuestas = pd.read_json("data/propuestas_26_04_2022.json", orient="records")
display(propuestas.shape)
propuestas.head()

(89, 12)

,title,link,resumen_de_la_propuesta,fecha_de_recepción,propuesto_por,estado,detalle,motivo_de_la_petición,beneficios_esperados,votos,lresumen_de_la_propuesta,
0,Proyecto conjunto de fachada (EN ESTUDIO),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=de9ecd81d3e50810VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Proyecto Conjunto de Fachada,Fecha de recepción: 01/04/2020,Propuesto por: Solicitante,Estado: EN ESTUDIO,"Detalle: Conocer si un edificio dispone de Proyecto Conjunto de Fachada, para poder acristalar las terrazas o cambiar el tipo de ventanas en edificios con fachadas protegidas.","Motivo de la petición: Cuando un ciudadano quiere acristalar una terraza o cambiar las ventanas en edificios con nivel 3 de protección o dentro del APE.00.01, es necesario un Proyecto Conjunto que defina la solución. En muchas ocasiones desconoces si la comunidad de propietario dispone de él o no, porque el presidente lo desconoce el administrador no les contesta... y lo solicitan mediante instancia general. Se podría ahorrar el trámite si esta información está publicada en la web.","Beneficios esperados: Se beneficiaría a todos los ciudadanos ofreciéndoles una información, y a la administración al ahorrar trámites de búsquedas de expedientes para verificar si existe o no dicho proyecto.",1,NaN,NaN
1,Registro de zonas básicas de salud confinadas por COVID 19 (NO COMPETENCIA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=d48fac5991340810VgnVCM2000001f4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Registro de zonas básicas de salud confinadas.,Fecha de recepción: 11/04/2020,Propuesto por: Solicitante,Estado: NO COMPETENCIA,Detalle: Base de datos que recoja las las fechas en que estuvieron confinadas las zonas básicas de salud de la Comunidad de Madrid.,Motivo de la petición: Estudio estadístico.,Beneficios esperados: Trabajo fin de grado.,7,NaN,NaN
2,Inventario de bienes municipales cedidos (PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=72d04ce679040810VgnVCM2000001f4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Inventario de bienes municipales cedidos,Fecha de recepción: 09/04/2022,Propuesto por: Enrique García Simón,Estado: PUBLICADA,"Detalle: Inventario de bienes con fecha cesión, duración, canon anual si aplica, CIF del adjudicatario, uso (educativo, religioso...)",Motivo de la petición: Transparencia.,Beneficios esperados: Publicidad para saber los bienes cedidos y en qué momento termina la cesión.,8,NaN,NaN
3,Red toma carga vehículos eléctricos en los 21 Distritos de Madrid-capital (PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=d73d006746c30810VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Red toma carga vehículos electricos en los 21 Distritos de Madrid-capital,Fecha de recepción: 8/04/2020,Propuesto por: Solicitante,Estado: PUBLICADA,Detalle: Estudiar la realización de una red de puntos carga para vehiculos eléctricos en los 21 Distritos de la capital.,Motivo de la petición: Facilitar la transición al vehículo eléctrico.,Beneficios esperados: Mejorar la calidad del aire de Madrid.,10,NaN,NaN
4,Vehículos que prestan o han prestado servicio en SAMUR Protección Civil (PARCIALMENTE PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=bc13bd2806eff710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Relación de vehículos que prestan o han prestado servicio en SAMUR Protección Civil.,Fecha de recepción: 03/04/2022,Propuesto por: Solicitan

In [12]:
propuestas.sort_values("votos", ascending=False).head()

,title,link,resumen_de_la_propuesta,fecha_de_recepción,propuesto_por,estado,detalle,motivo_de_la_petición,beneficios_esperados,votos,lresumen_de_la_propuesta,
6,Viviendas de Uso Turístico (PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=de4ba861d65de710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Viviendas de Uso Turístico,Fecha de recepción:05/02/2022,Propuesto por: Solicitante,Estado: PUBLICADA,Detalle: Localización de las viviendas de uso turístico aprobadas (con licencia) según el Plan Especial de Regulación del Uso de Servicios Terciarios en la clase de Hospedaje (PEH),Motivo de la petición: Poder saber qué viviendas en alquiler turístico son legales y cuales no.,Beneficios esperados: Disuasorio.,253,NaN,NaN
40,Criminalidad barrios de Madrid (NO COMPETENCIA / PARCIALMENTE PUBLICADA /NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=e47e8ca185b55710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Criminalidad barrios de Madrid,Fecha de recepción: 23/10/2020,Propuesto por: Solicitante,Estado: NO COMPETENCIA / PARCIALMENTE PUBLICADA /NO VIABLE,"Detalle: Estamos intentando hacer un mapa de la criminalidad en Madrid para poder desarrollar un programa de Criminalidad Ambiental para prevenir dichos delitos y para ello necesitamos conocer la criminalidad que tiene lugar en los distintos distritos de Madrid, concretamente de los barrios.",Motivo de la petición: Solicitamos esta petición porque no es posible acceder a los datos de otra manera y nos sería de gran utilidad.,Beneficios esperados: Con el conocimiento de estos datos esperamos poder realizar satisfactoriamente un proyecto que ayuda a prevenir los delitos en las zonas con mayor incidencia.,76,NaN,NaN
49,"Todos los arboles geolocalizados de Madrid contenidos en la plataforma ""un alcorque un arbol"" en formato GIS (shp, csv, kml...) (EN ESTUDIO)",https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=9163056583083710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,"Resumen de la propuesta: Todos los arboles geolocalizados de Madrid contenidos en la plataforma ""un alcorque un arbol"" en formato GIS (shp, csv, kml...) http://www-.munimadrid.es/DGPVE_WUAUA/irAMapa.do#",Fecha de recepción: 20/07/2020,Propuesto por: Solicitante,Estado: EN ESTUDIO,"Detalle: Todos los arboles geolocalizados de Madrid contenidos en la plataforma ""un alcorque un arbol"" en formato GIS.",Motivo de la petición: Estudio del arbolado en la ciudad de Madrid,Beneficios esperados: Trabajar con datos reales evitando hacer un inventario cuando ya existe uno.,72,NaN,NaN
45,Número de plazas de aparcamiento en superficie (PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=e111ed91bf504710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Número de plazas de aparcamiento en superficie,Fecha de recepción: 16/08/2020,Propuesto por: Solicitante,Estado: PUBLICADA,Detalle: Se trata de tener el censo de plazas de aparcamiento en superficie de la ciudad de Madrid.,Motivo de la petición: Se trata de poder evaluar el espacio público destinado al aparcamiento en la ciudad. Es notoria la ausencia de estos datos sobre todo en el conjunto de datos de la zona SER.,"Beneficios esperados: Conocimiento de plazas de aparcamiento y su impacto en los ingresos de la ciudad, así como la evaluación de la necesidad de construcción de aparcamientos.",49,NaN,NaN
48,Número vehículos privados por vivienda y barrio residente SER (PUBLICADA),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=d1ac056583083710VgnVCM1000001d4a900a

In [13]:
propuestas.groupby("estado").count()[["title"]].sort_values("title", ascending=False)

,title
estado,
Estado: EN ESTUDIO,18
Estado: PUBLICADA,15
Estado: NO VIABLE,14
Estado: PARCIALMENTE PUBLICADA,9
Estado: NO COMPETENCIA,5
Estado: PARCIALMENTE PUBLICADA,3
Estado: NO VIABLE,3
Estado: PARCIALMENTE PUBLICADA,2
Estado: PARCIALMENTE PUBLICADO,2


In [14]:
propuestas[propuestas["estado"].fillna("").str.contains("NO VIABLE")].sort_values("votos", ascending=False).head()

,title,link,resumen_de_la_propuesta,fecha_de_recepción,propuesto_por,estado,detalle,motivo_de_la_petición,beneficios_esperados,votos,lresumen_de_la_propuesta,
40,Criminalidad barrios de Madrid (NO COMPETENCIA / PARCIALMENTE PUBLICADA /NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=e47e8ca185b55710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Criminalidad barrios de Madrid,Fecha de recepción: 23/10/2020,Propuesto por: Solicitante,Estado: NO COMPETENCIA / PARCIALMENTE PUBLICADA /NO VIABLE,"Detalle: Estamos intentando hacer un mapa de la criminalidad en Madrid para poder desarrollar un programa de Criminalidad Ambiental para prevenir dichos delitos y para ello necesitamos conocer la criminalidad que tiene lugar en los distintos distritos de Madrid, concretamente de los barrios.",Motivo de la petición: Solicitamos esta petición porque no es posible acceder a los datos de otra manera y nos sería de gran utilidad.,Beneficios esperados: Con el conocimiento de estos datos esperamos poder realizar satisfactoriamente un proyecto que ayuda a prevenir los delitos en las zonas con mayor incidencia.,76,NaN,NaN
44,Coeficiente GINI (NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=c63e966a58d64710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Coeficiente GINI,Fecha de recepción: 07/09/2020,Propuesto por: Solicitante,Estado: NO VIABLE,Detalle: Coeficiente GINI de cada distrito de la ciudad de Madrid,Motivo de la petición: Proyecto de investigación con la Universidad de Alcalá de Henares.,NaN,38,NaN,NaN
30,ITE (Inspecciones Técnicas de Edificios) (NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=64b8f58157788710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: ITE (Inspecciones Técnicas de Edificios),Fecha de recepción: 30/03/2021,Propuesto por: Adriana Díaz,Estado: NO VIABLE,Detalle: Datos en formato CSV de las ITE desde 2022 hasta 2030.,"Motivo de la petición: Análisis urbano, estudiante en prácticas.",NaN,37,NaN,NaN
32,Residuos urbanos domésticos papel-cartón por Distrito (NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=a534c72d62a58710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Residuos urbanos domésticos papel-cartón por Distrito,Fecha de recepción: 21/03/2021,Propuesto por: Solicitante,Estado: NO VIABLE,"Detalle: La cantidad de toneladas de residuos de fraccion papel-cartón por distrito, al igual que se tienen el de envases, vidrio o resto. No entiendo porque no está el dato para papel-cartón cuando se tiene geolocalizado los contenedores azules para este tipo de residuo.",Motivo de la petición: Estudio recogida residuos solidos urbanos por fraccion/distrito/habitante/años,Beneficios esperados: Comprobar cantidad de residuos genrados por habitante en reogida selectiva y reciclados. Comprobar que se cumplen los objetivos de la directiva europea.. paro tenerlo por distrito de la ciudad de Madrid.,37,NaN,NaN
35,Número actual de viviendas con Licencia de Primera Ocupación en el ámbito urbanístico (NO VIABLE),https://datos.madrid.es/portal/site/egob/menuitem.3efdb29b813ad8241e830cc2a8a409a0/?vgnextoid=f479c7dc149a6710VgnVCM1000001d4a900aRCRD&vgnextchannel=102612b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default,Resumen de la propuesta: Número actual de viviendas con Licencia de Primera Ocupación en el ámbito urbanístico.,Fecha de recepción: 21/12/2020,Propuesto por: Solicitante,Estado: NO VIABLE,"Detalle: Se solicita conocer el número actualzado de viviendas con Licencia de Primera Ocupación de los ámbitos urbanísticos 